In [1]:
!wget -q --show-progress http://dl.caffe.berkeleyvision.org/bvlc_reference_caffenet.caffemodel -O bvlc_reference_caffenet.caffemodel

bvlc_reference_caff 100%[===================>] 232.56M  10.2MB/s    in 31s     


In [2]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2023-05-19 08:13:49--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2023-05-19 08:13:49 (42.3 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [3]:
!apt install -y caffe-cpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  blt caffe-tools-cpu cython3 fonts-lyx javascript-common libblosc1
  libcaffe-cpu1 libgflags2.2 libgoogle-glog0v5 libimagequant0 libjs-jquery
  libjs-jquery-ui liblbfgsb0 libleveldb1d liblzo2-2 python-matplotlib-data
  python-tables-data python3-atomicwrites python3-attr python3-backcall
  python3-blosc python3-bs4 python3-caffe-cpu python3-cloudpickle
  python3-cycler python3-dask python3-dateutil python3-decorator
  python3-et-xmlfile python3-fsspec python3-gflags python3-h5py
  python3-html5lib python3-imageio python3-importlib-metadata python3-ipython
  python3-ipython-genutils python3-jdcal python3-jedi python3-kiwisolver
  python3-locket python3-lxml python3-matplotlib python3-more-itertools
  python3-mpi4py python3-networkx python3-nose python3-numexpr python3-numpy
  python3-olefile python3-openpyxl python3-packaging python3-panda

In [8]:
# IMPORTS

import torch
import torch.nn as nn
import numpy as np
import caffe
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import urllib

In [5]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(    
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LocalResponseNorm(size=5),
            nn.Conv2d(96, 256, kernel_size=5, padding=2, groups=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LocalResponseNorm(size=5),
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1, groups=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1, groups=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(9216, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [6]:
net = AlexNet()
alexnet_weights = net.state_dict()
k = list(alexnet_weights.keys())

In [8]:
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)


input_image = Image.open(filename)

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
])


In [ ]:
caffe_net = caffe.Classifier("deploy_alexnet_updated.prototxt.txt", "bvlc_reference_caffenet.caffemodel",
                             mean = np.float32([104.0, 117.0, 123.0])) 
end = 'prob'
src = caffe_net.blobs['data']
w = caffe_net.blobs['data'].width
h = caffe_net.blobs['data'].height
input_tensor = 255*preprocess(input_image)-caffe_net.transformer.mean['data']

In [ ]:
src.reshape(1,3,h,w)
src.data[0] = input_tensor.numpy()
acts = caffe_net.forward(end=end)
probabilities = torch.tensor(acts[end][0])

# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

In [ ]:
def shai_net_to_py_readable(prototxt_filename, caffemodel_filename):
  net = caffe.Net(prototxt_filename, caffemodel_filename, caffe.TEST) # read the net + weights
  pynet_ = [] 
  for li in range(len(net.layers)):  # for each layer in the net
    layer = {}  # store layer's information
    layer['name'] = net._layer_names[li]
    # for each input to the layer (aka "bottom") store its name and shape
    layer['bottoms'] = [(net._blob_names[bi], net.blobs[net._blob_names[bi]].data.shape) 
                         for bi in list(net._bottom_ids(li))] 
    # for each output of the layer (aka "top") store its name and shape
    layer['tops'] = [(net ._blob_names[bi], net.blobs[net._blob_names[bi]].data.shape) 
                      for bi in list(net._top_ids(li))]
    layer['type'] = net.layers[li].type  # type of the layer
    # the internal parameters of the layer. not all layers has weights.
    layer['weights'] = [net.layers[li].blobs[bi].data[...] 
                        for bi in range(len(net.layers[li].blobs))]
    pynet_.append(layer)
  return pynet_

In [ ]:
res = shai_net_to_py_readable("deploy_alexnet_updated.prototxt.txt", ".caffemodel")
sub_res = []
for i in range(len(res)):
  if len(res[i]['weights']) > 0:
    sub_res.append(res[i]['weights'][0])
    sub_res.append(res[i]['weights'][1])
for i in range(16):
  assert alexnet_weights[k[i]].shape == torch.tensor(sub_res[i]).shape, "error"
  alexnet_weights[k[i]] = torch.tensor(sub_res[i])
k = list(alexnet_weights.keys())


In [ ]:
net.load_state_dict(alexnet_weights)

In [ ]:
net.eval()
with torch.no_grad():
  r = net(input_tensor.unsqueeze(0))
probabilities = torch.nn.functional.softmax(r[0], dim=0)
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

In [ ]:
torch.save(net.state_dict(), "net.pt")

#TESTING

In [ ]:
net = AlexNet()
net.load_state_dict(alexnet_weights)
hook = net.features[0].register_forward_hook(get)
hook.remove()



In [ ]:
caffe_net = caffe.Classifier("deploy_alexnet_updated.prototxt.txt", "bvlc_reference_caffenet.caffemodel",
                             mean = np.float32([104.0, 117.0, 123.0])) 

src = caffe_net.blobs['data']
w = caffe_net.blobs['data'].width
h = caffe_net.blobs['data'].height

src.reshape(1,3,h,w)
src.data[0] = input_tensor.numpy()

acts = caffe_net.forward(end='conv1')
probabilities = torch.tensor(acts[end][0])


# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())
